In [1]:
import numpy as np 
import pandas as pd
import os
import scipy.stats as ss
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error,mean_squared_error
from sklearn.model_selection import GridSearchCV
from scipy.stats import f_oneway
import pandas as pd
import datetime
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
  except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
    print(e)

In [3]:
# 데이터 불러오기
past_sales = pd.read_csv('C:/Users/rlaeo/OneDrive/바탕 화면/rossmann.csv')
store = pd.read_csv('C:/Users/rlaeo/OneDrive/바탕 화면/store.csv')

# 데이터 제거
store = store.drop(['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'], axis=1)

# 데이터 병합
data = pd.merge(left = past_sales,right = store, on = 'Store', how = 'right')

# 날짜
data['Date'] = pd.to_datetime(data['Date'], format="%Y-%m-%d")
data['Year']=pd.DatetimeIndex(data.Date).year
data['Month']=pd.DatetimeIndex(data.Date).month
data['Day']=pd.DatetimeIndex(data.Date).day
data = data.drop(['Date'],axis=1)

# 데이터 가공 : StateHoliday, Open
data['StateHoliday'] = data['StateHoliday'].replace({0:'d'}) # 0 값은 d로 대체
data['StateHoliday'] = data['StateHoliday'].replace({'0':'d'})# 0 값은 d로 대체
open = data[data['Open']==0].index
data.drop(open,inplace=True)


# 결측값 처리
data = data.dropna(axis=0)

# store,open 이름 제거
data = data.drop(['Open'],axis=1)
digital_cols = ['Customers', 'CompetitionDistance', 'Year','Month', 'Day']
digital_data = data[digital_cols]

onehotencoding_cols = ['StateHoliday', 'StoreType', 'Assortment', 'DayOfWeek']
categorey_data = pd.get_dummies(data=data, columns=onehotencoding_cols)
data = pd.concat([digital_data, categorey_data], axis = 1)
data = data.astype({'Promo2':'int', 'StateHoliday_b':'int', 'StateHoliday_c':'int', 'StateHoliday_d':'int', 'StoreType_a':'int', 'StoreType_b':'int', 'StoreType_c':'int', 'StoreType_d':'int'})
data = data.astype({'Assortment_a':'int', 'Assortment_b':'int', 'Assortment_c':'int', 'DayOfWeek_1':'int', 'DayOfWeek_2':'int', 'DayOfWeek_3':'int', 'DayOfWeek_4':'int', 'DayOfWeek_5':'int', 'DayOfWeek_6':'int', 'DayOfWeek_7':'int'})
pd.options.display.max_columns = None
data['StateHoliday_a'] = data['StateHoliday_a'].replace({True: 1, False: 0})
data = data.drop(['SchoolHoliday'],axis=1)
data

C:\Users\rlaeo\AppData\Local\Temp\ipykernel_7220\47868744.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  past_sales = pd.read_csv('C:/Users/rlaeo/OneDrive/바탕 화면/rossmann.csv')


,Customers,CompetitionDistance,Year,Month,Day,Store,Sales,Customers,Promo,CompetitionDistance,Promo2,Year,Month,Day,StateHoliday_a,StateHoliday_b,StateHoliday_c,StateHoliday_d,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,DayOfWeek_1,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7
0,555,1270.0,2015,7,31,1,5263,555,1,1270.0,0,2015,7,31,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0
1,546,1270.0,2015,7,30,1,5020,546,1,1270.0,0,2015,7,30,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0
2,523,1270.0,2015,7,29,1,4782,523,1,1270.0,0,2015,7,29,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0
3,560,1270.0,2015,7,28,1,5011,560,1,1270.0,0,2015,7,28,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0
4,612,1270.0,2015,7,27,1,6102,612,1,1270.0,0,2015,7,27,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017202,471,5350.0,2013,1,7,1115,6905,471,1,5350.0,1,2013,1,7,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0
1017204,339,5350.0,2013,1,5,1115,4771,339,0,5350.0,1,2013,1,5,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0
1017205,326,5350.0,2013,1,4,1115,4540,326,0,5350.0,1,2013,1,4,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0
1017206,300,5350.0,2013,1,3,1115,4297,300,0,5350.0,1,2013,1,3,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0


In [7]:
data = data.groupby('Store').get_group(610)
data

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3706641500.py, line 1)

In [5]:
target_col = 'Sales'
features_cols = data.columns.drop(target_col)        
features = data[features_cols]
target = data[target_col]
target = pd.DataFrame(target.values.reshape(-1,1))
features = features.values
target = target.values
seed = 0
np.random.seed(seed)
tf.compat.v1.set_random_seed(seed)
print(features.shape)
print(target.shape)

(779, 41)
(779, 1)


In [6]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=seed,shuffle=True)
print('train_data의 row : %s, column : %s'%(x_train.shape[0],x_train.shape[1]))
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

train_data의 row : 623, column : 41
